Importe bibliotecas necessárias

In [44]:
import pandas as pd
from IPython.display import display, HTML
from ipywidgets import Button, HBox, VBox,widgets, HTML
from fpdf import FPDF
import time
import matplotlib.pyplot as plt
import dataframe_image as dfi
import plotly.express as px

Parâmetros:

In [45]:
# Lista de atacantes
atacantes = ['TT', 'Balula', 'Bruno','Vini', 'Mano Brown', 'Cavalo']

# Lista de origens do ataque
origens = ['Entrada', 'Meio', 'Saída', 'Fundo Meio', 'Fundo Saída']

# Lista de números de posição
destinos = ['Diagonal','Diagonal Curta','Paralela','Positiva','Negativa']

# Lista de descrições dos tipos de ataques
descricoes_tipos_ataque = ['Kill', 'Voltou', 'Pinga', 'Erro']

# Titulo do relatório PDF
TITULO_RELATORIO = 'Relatório de Ataques - POLI X PINHEIROS'

# Data do Jogo:
DATA_JOGO = '28/04/24'

# Resultado do Jogo
RESULTADO_JOGO = 'POLI 1 X 2 PINHEIROS'

Reseta o Dataframe que armazena os jogos:

In [46]:
df = pd.DataFrame(columns=['Atacante', 'Tipo', 'Origem', 'Destino'])

Cria os botões:
Aperte em um botão de cada coluna (necessariamente) na ordem (necessariamente) da esquerda para a direita. Caso erre, aperte reset para dar clear no vetor 'row'. Save and Reset salva o vetor 'row' no dataframe e dá clear nele.

In [51]:
row = []

def create_button(name):
    button = widgets.Button(description=name)
    def onclick(b):
        row.append(name)
    button.on_click(onclick)
    return button

# Lista de botões para cada atacante
botoes_atacantes = [create_button(atacante) for atacante in atacantes]

# Lista de botões para cada origem do ataque
botoes_origem = [create_button(origem) for origem in origens]

# Lista de botões para cada posição de destino
botoes_destino = [create_button(numero) for numero in destinos]

# Lista de botões para cada tipo de ataque
botoes_tipos_ataque = [create_button(descricao) for descricao in descricoes_tipos_ataque]


# Save and reset buttons
reset = widgets.Button(description='Reset')
def reset_func(b):
    row.clear()
reset.on_click(reset_func)

save_reset = widgets.Button(description='Save and Reset')
def save_reset_func(b):
    df.loc[len(df)] = row
    row.clear()
save_reset.on_click(save_reset_func)

# Definir o layout dos botões
coluna_botoes1 = VBox([HTML(value='<b>Atacante</b>'), *botoes_atacantes])
coluna_botoes2 = VBox([HTML(value='<b>Origem</b>'), *botoes_origem])
coluna_botoes3 = VBox([HTML(value='<b>Destino</b>'), *botoes_destino])
coluna_botoes4 = VBox([HTML(value='<b>Tipo</b>'), *botoes_tipos_ataque])

# Criar o layout final com três colunas
layout = HBox([coluna_botoes1, coluna_botoes4, coluna_botoes2, coluna_botoes3])

# Colocar botão de save e reset embaixo
layout = VBox([layout, HBox([reset,save_reset])])

display(layout)

Verifique como está o dataframe:

In [15]:
df

,Atacante,Tipo,Origem,Destino
0,TT,Kill,Entrada,Paralela
1,Balula,Erro,Fundo Meio,Negativa
2,TT,Erro,Entrada,Diagonal
3,TT,Voltou,Entrada,Diagonal
4,TT,Erro,Entrada,Diagonal
5,TT,Voltou,Entrada,Diagonal
6,TT,Voltou,Entrada,Diagonal
7,TT,Voltou,Entrada,Diagonal
8,Bruno,Kill,Entrada,Diagonal
9,Mano Brown,Voltou,Meio,Negativa


Se quiser, salve este dataframe em csv:

In [16]:
df.to_csv('set3.csv')

Unindo os 3 dataframes em 1 só:

In [54]:
df1 = pd.read_csv('set1.csv')
df2 = pd.read_csv('set2.csv')
df3 = pd.read_csv('set3.csv')
df = pd.concat([df1, df2, df3], ignore_index=True)
df = df.drop(columns=['Unnamed: 0'])
df.to_csv('all_sets.csv')
df

,Atacante,Tipo,Origem,Destino
0,Vini,Voltou,Saída,Diagonal
1,Mano Brown,Kill,Meio,Negativa
2,Balula,Voltou,Entrada,Diagonal
3,TT,Kill,Entrada,Diagonal
4,Mano Brown,Voltou,Meio,Positiva
...,...,...,...,...
75,TT,Kill,Entrada,Paralela
76,TT,Kill,Entrada,Diagonal Curta
77,Bruno,Pinga,Entrada,Paralela
78,TT,Voltou,Fundo Meio,Negativa


Funções que criam os plots do relatório

In [55]:
def plot_atacante():
    # Crie um plot de barras com a quantidade de ataques de cada jogador com plotly
    fig = px.bar(df['Atacante'].value_counts(), x=df['Atacante'].value_counts().index, y=df['Atacante'].value_counts().values)
    fig.update_layout(
    xaxis_title="Atacante",
    yaxis_title="Número de Ataques",
    #color=df['Atacante'].value_counts().index,
    yaxis=dict(
        tickmode='linear',  # Apenas mostrar escala inteira no eixo Y
        dtick=1  # Incremento de 1 na escala do eixo Y
    )
    )
    # Salve a imagem do plot
    fig.write_image('fig1.png')

def plot_origem():
    # Crie um plot com a quantidade de ataques de origem
    fig2 = px.bar(df['Origem'].value_counts(), x=df['Origem'].value_counts().index, y=df['Origem'].value_counts().values)
    fig2.update_layout(
    xaxis_title="Origem dos Ataques",
    yaxis_title="Número de Ataques",
    #color=df['Origem'].value_counts().index,
    yaxis=dict(
        tickmode='linear',  # Apenas mostrar escala inteira no eixo Y
        dtick=1  # Incremento de 1 na escala do eixo Y
    )
    )
    # Salve a imagem do plot
    fig2.write_image('fig2.png')

def plot_destino():
    # Crie um plot com a quantidade de ataques de destino
    fig3 = px.bar(df['Destino'].value_counts(), x=df['Destino'].value_counts().index, y=df['Destino'].value_counts().values)
    fig3.update_layout(
    xaxis_title="Destino dos Ataques",
    yaxis_title="Número de Ataques",
    #color=df['Destino'].value_counts().index,
    yaxis=dict(
        tickmode='linear',  # Apenas mostrar escala inteira no eixo Y
        dtick=1  # Incremento de 1 na escala do eixo Y
    )
    )
    # Salve a imagem do plot
    fig3.write_image('fig3.png')

def plot_tipo():
    # Crie um plot com a quantidade de ataques de cada tipo
    fig4 = px.bar(df['Tipo'].value_counts(), x=df['Tipo'].value_counts().index, y=df['Tipo'].value_counts().values)
    fig4.update_layout(
    xaxis_title="Tipos de Ataques",
    yaxis_title="Número de Ataques",
    #color=df['Tipo'].value_counts().index,
    yaxis=dict(
        tickmode='linear',  # Apenas mostrar escala inteira no eixo Y
        dtick=1  # Incremento de 1 na escala do eixo Y
    )
    )
    # Salve a imagem do plot
    fig4.write_image('fig4.png')

# PLote o número de origens por cada jogador
def plot_origem_atacante(atacante):
    fig5 = px.bar(df[df['Atacante'] == atacante]['Origem'].value_counts(), x=df[df['Atacante'] == atacante]['Origem'].value_counts().index, y=df[df['Atacante'] == atacante]['Origem'].value_counts().values)
    fig5.update_layout(
    xaxis_title="Origem dos Ataques",
    yaxis_title="Número de Ataques",
    #color=df['Origem'].value_counts().index,
    yaxis=dict(
        tickmode='linear',  # Apenas mostrar escala inteira no eixo Y
        dtick=1  # Incremento de 1 na escala do eixo Y
    )
    )
    fig5.write_image(f'fig5_{atacante}.png')

# PLote o número de destinos por cada jogador
def plot_destino_atacante(atacante):
    fig6 = px.bar(df[df['Atacante'] == atacante]['Destino'].value_counts(), x=df[df['Atacante'] == atacante]['Destino'].value_counts().index, y=df[df['Atacante'] == atacante]['Destino'].value_counts().values)
    fig6.update_layout(
    xaxis_title="Destino dos Ataques",
    yaxis_title="Número de Ataques",
    #color=df['Destino'].value_counts().index,
    yaxis=dict(
        tickmode='linear',  # Apenas mostrar escala inteira no eixo Y
        dtick=1  # Incremento de 1 na escala do eixo Y
    )
    )
    fig6.write_image(f'fig6_{atacante}.png')

# PLote o número de tipos por cada jogador
def plot_tipo_atacante(atacante):
    fig7 = px.bar(df[df['Atacante'] == atacante]['Tipo'].value_counts(), x=df[df['Atacante'] == atacante]['Tipo'].value_counts().index, y=df[df['Atacante'] == atacante]['Tipo'].value_counts().values)
    fig7.update_layout(
    xaxis_title="Tipos de Ataques",
    yaxis_title="Número de Ataques",
    #color=df['Tipo'].value_counts().index,
    yaxis=dict(
        tickmode='linear',  # Apenas mostrar escala inteira no eixo Y
        dtick=1  # Incremento de 1 na escala do eixo Y
    )
    )
    fig7.write_image(f'fig7_{atacante}.png')

Funções que auxiliam na criação do relatório:

In [39]:
def create_letterhead(pdf, WIDTH):
    pdf.image("./resources/heicoders_letterhead_cropped.png", 0, 0, WIDTH)

def create_title(title, pdf):

    # Add main title
    pdf.set_font('Times', 'B', 20)
    pdf.ln(40)
    pdf.write(5, title)
    pdf.ln(10)

    # Add date of report
    pdf.set_font('Times', 'B', 14)
    pdf.set_text_color(r=128,g=128,b=128)
    pdf.write(4, DATA_JOGO)

    # Add line break
    pdf.ln(10)

def write_to_pdf(pdf, words):

    # Set text colour, font size, and font type
    pdf.set_text_color(r=0,g=0,b=0)
    pdf.set_font('Times', 'B', 12)

    pdf.write(5, words)

class PDF(FPDF):

    def footer(self):
        self.set_y(-15)
        self.set_font('Times', 'I', 8)
        self.set_text_color(128)
        self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')

Criação do Relatório:

In [56]:
# Global Variables
WIDTH = 210
HEIGHT = 297

# Carregando as imagens dos plots
plot_atacante()
plot_origem()
plot_destino()
plot_tipo()

# Create PDF
pdf = PDF() # A4 (210 by 297 mm)
'''
First Page of PDF
'''
# Add Page
pdf.add_page()

# Add lettterhead and title
create_title(TITULO_RELATORIO, pdf)

write_to_pdf(pdf, f'Resultado do Jogo: {RESULTADO_JOGO}')
pdf.ln(10)

write_to_pdf(pdf, "Atacantes do Jogo:")
pdf.ln(7)

i=1
for atacante in atacantes:
    write_to_pdf(pdf, f'{i}: {atacante}')
    pdf.ln(5)
    i+=1

pdf.ln(7)

# Add some words to PDF
write_to_pdf(pdf, "Número de ataques por jogador:")
pdf.ln(7)

# Add table
pdf.image("fig1.png", w=170)
pdf.ln(10)



'''
Second Page of PDF
'''

# Add Page
pdf.add_page()

# Add some words to PDF
write_to_pdf(pdf, "Número de ataques por origem:")
pdf.ln(7)

# Add the generated visualisations to the PDF
pdf.image("fig2.png", w=170)
pdf.ln(10)

'''
Third Page of PDF
'''

# Add Page
pdf.add_page()

# Add some words to PDF
write_to_pdf(pdf, "Número de ataques por destino:")
pdf.ln(7)

# Add the generated visualisations to the PDF
pdf.image("fig3.png", w=170)
pdf.ln(10)

'''
Fourth Page of PDF
'''

# Add Page
pdf.add_page()

# Add some words to PDF
write_to_pdf(pdf, "Número de ataques por tipo:")
pdf.ln(7)

# Add the generated visualisations to the PDF
pdf.image("fig4.png", w=170)
pdf.ln(10)

'''
Loop para relatórios Individuais
'''

for atacante in atacantes:
    # Carregando pngs
    plot_origem_atacante(atacante)
    plot_destino_atacante(atacante)
    plot_tipo_atacante(atacante)

    # Add Page
    pdf.add_page()

    # Add some words to PDF
    write_to_pdf(pdf, f"Número de ataques por origem para o jogador {atacante}:")
    pdf.ln(7)

    # Add the generated visualisations to the PDF
    pdf.image(f"fig5_{atacante}.png", w=170)

    # Add Page
    pdf.add_page()

    # Add some words to PDF
    write_to_pdf(pdf, f"Número de ataques por destino para o jogador {atacante}:")
    pdf.ln(7)

    # Add the generated visualisations to the PDF
    pdf.image(f"fig6_{atacante}.png", w=170)

    # Add Page
    pdf.add_page()

    # Add some words to PDF
    write_to_pdf(pdf, f"Número de ataques por tipo para o jogador {atacante}:")
    pdf.ln(7)

    # Add the generated visualisations to the PDF
    pdf.image(f"fig7_{atacante}.png", w=170)


# Generate the PDF
pdf.output("relatorio.pdf", 'F')

''